# Сессия 2

## ВВЕДЕНИЕ:
На этой сессии, я буду продолжать исследования, опираясь на то, что я уже разработал. Мне необходимо выполнить предварительную обработку данных, найти ключевые слова, биграммы/триграммы для их дальнейшего использования в построении моделей. 


---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re

Чтение документа

In [2]:
df = pd.read_csv("df.csv")

Реиндексация

In [3]:
df.reset_index(drop=True, inplace=True)

In [4]:
df

,district_name,sidebar.id,payload.dt,sidebar.author_name,sidebar.latitude,sidebar.longitude,sidebar.full_address,sidebar.responsible.data.executor.organization_name,sidebar.responsible.data.executor.person_position,sidebar.responsible.data.executor.person_name,sidebar.responsible.data.executor.person_phone,sidebar.responsible.data.executor.person_email,sidebar.municipality.name,reason.name,reason.category.city_object.name,payload.body_answer,reason.category.name,appeal,clean_appeal,clean_sentence
0,Приморский,2947909,2020-10-22 16:33:46.128707,Иванов А.,59.994334,30.197107,"г.Санкт-Петербург, Мебельная улица, дом 47, ко...",Администрация Приморского района,Заместитель главы администрации,Гырла Артем Ульянович,576-82-14,zhilprim@tuprim.gov.spb.ru,муниципальный округ №65,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\n\r\nНастоящим сообщ...,Содержание МКД,"1й подъезд, 5 этаж. В коридоре над XXXX виси...","['подъезд', 'коридор', 'висеть', 'какойтый', '...",подъезд коридор висеть какойтый провод возможн...
1,Колпинский,2947918,2020-11-05 09:57:37.568325,Ольга Т.,59.735262,30.570192,"город Колпино, улица Машиностроителей, дом 12,...",СПб ГКУ ЖА Колпинского района,Заместитель директора СПб ГКУ ЖА Колпинского р...,Иванов Владислав Николаевич,461-56-60,guzacentr@yandex.ru,город Колпино,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\n\r\nРассмотрев Ваше...,Содержание МКД,Маленькмй лифт изрисован внутри.,"['маленькмть', 'изрисовать', 'внутри']",маленькмть изрисовать внутри
2,Красносельский,2947923,2020-11-03 09:04:00.828932,Надежда Ф.,59.733110,30.087842,Неизвестно,"СПб ГКУ ""Дирекция по организации дорожного дви...",Заместитель директора,Зайченков Сергей Игоревич,241-2586,info@gudodd.ru,город Красное Село,Несанкционированные надписи / объявления на те...,Улица,Санкт-Петербургское государственное казенное у...,Благоустройство,реклама на светофоре,"['реклама', 'светофор']",реклама светофор
3,Приморский,2947965,2020-10-30 16:57:19.289164,Иванов А.,59.994334,30.197107,"г.Санкт-Петербург, Мебельная улица, дом 47, ко...",Администрация Приморского района,Заместитель главы администрации,Гырла Артем Ульянович,576-82-14,zhilprim@tuprim.gov.spb.ru,муниципальный округ №65,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\nРассмотрев Ваше ...,Содержание МКД,"1й подъезд, 5 этаж, квартирный холл. Рааботник...","['подъезд', 'квартирный', 'рааботник', 'ленинг...",подъезд квартирный рааботник ленинградский веч...
4,Адмиралтейский,2947973,2020-12-16 09:35:24.195742,Алекса Р.,59.911521,30.269076,"г.Санкт-Петербург, Курляндская улица, дом 47, ...",Администрация Адмиралтейского района,Заместитель главы администрации,Степанов Алексей Викторович,576-97-71,stepanov@tuadm.gov.spb.ru,муниципальный округ Екатерингофский,Самовольное размещение наружн. блоков систем к...,Дом,Уважаемый пользователь!\r\n\r\nРассмотрев Ваше...,Нарушение правил пользования общим имуществом,незаконная решетка на фасаде,"['незаконный', 'решётка', 'фасад']",незаконный решётка фасад
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60876,Невский,3375243,2021-04-22 17:10:28.180799,Антон П.,59.881085,30.438858,"г.Санкт-Петербург, улица Бабушкина, дом 36, ли...",СПб ГКУ ЖА Невского района,Директор,Середин Владимир Ипполитович,246-08-08,guja_nev@mail.ru,муниципальный округ Ивановский,Отсутствие или несоблюдение графика уборки под...,Дом,Уважаемый пользователь!\n\nРассмотрев Ваше соо...,Содержание МКД,Подъезд не убирается! График уборки не соблюда...,"['подъезд', 'убираться', 'график', 'уборка', '...",подъезд убираться график уборка соблюдаться уб...
60877,Центральный,3375329,2021-04-23 09:43:33.320180,Антон К.,59.941825,30.323319,Неизвестно,Администрация Центрального района,Заместитель главы администрации,Долгополый Михаил Николаевич,576-28-00,dolgopolyi@tucentr.gov.spb.ru,муниципальный округ Дворцовый округ,Незаконное размещение информационной и (или) р...,Территория Санкт-Петербурга,Размещение

### 2.1. Предварительная обработка данных 

Импорт необходимых библиотек

In [5]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2

Для тестов вытащу одну жалобу

In [6]:
#Использовал iloc что бы вытащить 0 строку по индексу
sentence_example = df.iloc[0]["appeal"]

Обьявление констант для функции которая описана ниже

In [7]:
#MorphAnalyzer нужен для преведения слова в начальную фонрму (леметизация)
morph = pymorphy2.MorphAnalyzer()
#stopwords нудна для удаления предлогов и прочего мусора который создает шум и ухудшает качество алгоритмов
russian_stop_words = stopwords.words("russian")

Теперь нужна функция, которую потом можно будет сунуть в apply

In [8]:
#Функция будет принимтаь на вход строку и будевое значение (может возникнуть ситуация в которой будут не нужны стоп слова)
def tokinize_sentence(sentense: str, remove_stop_word: bool = True):
    #Первый этап - разделения предложения на токены или же отдельные кусочки
    tokens = word_tokenize(sentense, language="russian")
    #Далее необходимо убрать пунктуацию тк как они не нужна и создает много шумов
    tokens = [i for i in tokens if i not in string.punctuation]
    #Теперь удаление стоп слов
    if remove_stop_word:
        tokens = [i for i in tokens if i not in russian_stop_words]
    
    #В жалобах появляется много непонятных Х по этому от них тоже необходимо избавится
    stammed_tokens_origin = []
    for i in tokens:
        #запишу в отдельный список все слова которые не содержат Х
        if 'X' not in i:
            stammed_tokens_origin.append(i)
    #Переприсвоил список для удобства
    tokens = stammed_tokens_origin
    
    #А теперь вернем словам их первоначальную форму
    tokens = [morph.parse(i)[0].normal_form for i in tokens]
    
    return tokens

Удалю все спец символы и цифры, что бы остались только слова

In [9]:
#Удалю все спец символы и цифры при помощи регулярных выражений, а strip удалит лишние пробелы в начале и в конце предложения
df["clean_appeal"] = df["appeal"].apply(lambda x: re.compile('[^а-яА-Я ]').sub('', x)) 
#Теперь применю вышеописанную функцию к каждому элементу атрибьута appeal
df["clean_appeal"] = df["clean_appeal"].apply(lambda x: tokinize_sentence(x, remove_stop_word=True))

Решил так же удалить все значения списка у которых менее 3х символов (сложно догадаться что имелось ввиду когда видешь "пра" или "й")

In [10]:
#Удаление огрызков слов
df["clean_appeal"] = df["clean_appeal"].apply(lambda x: [i for i in x if len(i) > 4])

Теперь соеденю все слова в предложения и помещу в отдельный атрибут

In [11]:
#Для этой зажачи вновь использую метод apply
df["clean_sentence"] = df["clean_appeal"].apply(lambda x:' '.join(x))

In [12]:
df

,district_name,sidebar.id,payload.dt,sidebar.author_name,sidebar.latitude,sidebar.longitude,sidebar.full_address,sidebar.responsible.data.executor.organization_name,sidebar.responsible.data.executor.person_position,sidebar.responsible.data.executor.person_name,sidebar.responsible.data.executor.person_phone,sidebar.responsible.data.executor.person_email,sidebar.municipality.name,reason.name,reason.category.city_object.name,payload.body_answer,reason.category.name,appeal,clean_appeal,clean_sentence
0,Приморский,2947909,2020-10-22 16:33:46.128707,Иванов А.,59.994334,30.197107,"г.Санкт-Петербург, Мебельная улица, дом 47, ко...",Администрация Приморского района,Заместитель главы администрации,Гырла Артем Ульянович,576-82-14,zhilprim@tuprim.gov.spb.ru,муниципальный округ №65,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\n\r\nНастоящим сообщ...,Содержание МКД,"1й подъезд, 5 этаж. В коридоре над XXXX виси...","[подъезд, коридор, висеть, какойтый, провод, в...",подъезд коридор висеть какойтый провод возможн...
1,Колпинский,2947918,2020-11-05 09:57:37.568325,Ольга Т.,59.735262,30.570192,"город Колпино, улица Машиностроителей, дом 12,...",СПб ГКУ ЖА Колпинского района,Заместитель директора СПб ГКУ ЖА Колпинского р...,Иванов Владислав Николаевич,461-56-60,guzacentr@yandex.ru,город Колпино,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\n\r\nРассмотрев Ваше...,Содержание МКД,Маленькмй лифт изрисован внутри.,"[маленькмть, изрисовать, внутри]",маленькмть изрисовать внутри
2,Красносельский,2947923,2020-11-03 09:04:00.828932,Надежда Ф.,59.733110,30.087842,Неизвестно,"СПб ГКУ ""Дирекция по организации дорожного дви...",Заместитель директора,Зайченков Сергей Игоревич,241-2586,info@gudodd.ru,город Красное Село,Несанкционированные надписи / объявления на те...,Улица,Санкт-Петербургское государственное казенное у...,Благоустройство,реклама на светофоре,"[реклама, светофор]",реклама светофор
3,Приморский,2947965,2020-10-30 16:57:19.289164,Иванов А.,59.994334,30.197107,"г.Санкт-Петербург, Мебельная улица, дом 47, ко...",Администрация Приморского района,Заместитель главы администрации,Гырла Артем Ульянович,576-82-14,zhilprim@tuprim.gov.spb.ru,муниципальный округ №65,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\nРассмотрев Ваше ...,Содержание МКД,"1й подъезд, 5 этаж, квартирный холл. Рааботник...","[подъезд, квартирный, рааботник, ленинградский...",подъезд квартирный рааботник ленинградский веч...
4,Адмиралтейский,2947973,2020-12-16 09:35:24.195742,Алекса Р.,59.911521,30.269076,"г.Санкт-Петербург, Курляндская улица, дом 47, ...",Администрация Адмиралтейского района,Заместитель главы администрации,Степанов Алексей Викторович,576-97-71,stepanov@tuadm.gov.spb.ru,муниципальный округ Екатерингофский,Самовольное размещение наружн. блоков систем к...,Дом,Уважаемый пользователь!\r\n\r\nРассмотрев Ваше...,Нарушение правил пользования общим имуществом,незаконная решетка на фасаде,"[незаконный, решётка, фасад]",незаконный решётка фасад
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60876,Невский,3375243,2021-04-22 17:10:28.180799,Антон П.,59.881085,30.438858,"г.Санкт-Петербург, улица Бабушкина, дом 36, ли...",СПб ГКУ ЖА Невского района,Директор,Середин Владимир Ипполитович,246-08-08,guja_nev@mail.ru,муниципальный округ Ивановский,Отсутствие или несоблюдение графика уборки под...,Дом,Уважаемый пользователь!\n\nРассмотрев Ваше соо...,Содержание МКД,Подъезд не убирается! График уборки не соблюда...,"[подъезд, убираться, график, уборка, соблюдать...",подъезд убираться график уборка соблюдаться уб...
60877,Центральный,3375329,2021-04-23 09:43:33.320180,Антон К.,59.941825,30.323319,Неизвестно,Администрация Центрального района,Заместитель главы администрации,Долгополый Михаил Николаевич,576-28-00,dolgopolyi@tucentr.gov.spb.ru,муниципальный округ Дворцовый округ,Незаконное размещение информационной и (или) р...,Территория Санкт-Петербурга,Размещение наружной реклам

### 2.2. Поиск ключевых слов/биграмм/триграмм

## Алгоритмы нахождения ключевых слов
Рассмотрим несколько алгоритмов для нахождения ключевых слов
### TfidfVectorizer
Преобразует набор необработанных документов в матрицу функций TF-IDF
### HashingVectorizer
Преобразует коллекции текстовых документов в матрицу вхождений токенов
### CountVectorizer
Преобразует коллекцию текстовых документов в матрицу счетчиков токенов

Обоснование выбора алгоритма: я выбираю алгоритм `TfidfVectorizer`, поскольку он способен определять важность слов или ngram в векторном формате разреженной матрицы, что способно повысить качество определения важности(они же ключевые слова или ngrma'ы) таких ngram

### Образование новых признаков с ключевыми словами
Ниже представленны методы, которые формируют атрибуты с unigram, bigram и trigram. Также есть возможность получения лучшей ngramm в поле с помощью метрики tf-idf в словаре поля, если это необходимо 

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(min_df=15, max_df=0.95, ngram_range=(1,1))
X = vectorizer.fit_transform(df['clean_sentence'])
vect_unigram = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
vect_unigram.head()

,абрамов,абсолютно,авангардный,аварийка,аварийность,аварийный,авария,август,авиаконструктор,автобус,...,этажотсутствовать,эффект,южный,юридический,являться,якобы,ямочный,январь,ярослав,яхтенный
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Биграмм

In [15]:
vectorizer = TfidfVectorizer(min_df=15, max_df=0.95, ngram_range=(1,2))
X = vectorizer.fit_transform(df['clean_sentence'])
vect_begram = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
vect_begram.head()

,абрамов,абсолютно,авангардный,аварийка,аварийность,аварийный,аварийный ситуация,аварийный служба,аварийный состояние,авария,...,являться нарушение,являться неактуальный,якобы,ямочный,ямочный ремонт,январь,январь управлять,ярослав,ярослав гашек,яхтенный
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Триграмм

In [16]:
vectorizer = TfidfVectorizer(min_df=15, max_df=0.95, ngram_range=(1,3))
X = vectorizer.fit_transform(df['clean_sentence'])
vect_trigram = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
vect_trigram.head()

,абрамов,абсолютно,авангардный,аварийка,аварийность,аварийный,аварийный ситуация,аварийный служба,аварийный состояние,авария,...,якобы,ямочный,ямочный ремонт,ямочный ремонт латка,январь,январь управлять,январь управлять организация,ярослав,ярослав гашек,яхтенный
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df2 = pd.concat([df, vect_unigram, vect_begram, vect_trigram], axis=1)

In [18]:
df2

,district_name,sidebar.id,payload.dt,sidebar.author_name,sidebar.latitude,sidebar.longitude,sidebar.full_address,sidebar.responsible.data.executor.organization_name,sidebar.responsible.data.executor.person_position,sidebar.responsible.data.executor.person_name,...,якобы,ямочный,ямочный ремонт,ямочный ремонт латка,январь,январь управлять,январь управлять организация,ярослав,ярослав гашек,яхтенный
0,Приморский,2947909,2020-10-22 16:33:46.128707,Иванов А.,59.994334,30.197107,"г.Санкт-Петербург, Мебельная улица, дом 47, ко...",Администрация Приморского района,Заместитель главы администрации,Гырла Артем Ульянович,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Колпинский,2947918,2020-11-05 09:57:37.568325,Ольга Т.,59.735262,30.570192,"город Колпино, улица Машиностроителей, дом 12,...",СПб ГКУ ЖА Колпинского района,Заместитель директора СПб ГКУ ЖА Колпинского р...,Иванов Владислав Николаевич,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Красносельский,2947923,2020-11-03 09:04:00.828932,Надежда Ф.,59.733110,30.087842,Неизвестно,"СПб ГКУ ""Дирекция по организации дорожного дви...",Заместитель директора,Зайченков Сергей Игоревич,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Приморский,2947965,2020-10-30 16:57:19.289164,Иванов А.,59.994334,30.197107,"г.Санкт-Петербург, Мебельная улица, дом 47, ко...",Администрация Приморского района,Заместитель главы администрации,Гырла Артем Ульянович,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Адмиралтейский,2947973,2020-12-16 09:35:24.195742,Алекса Р.,59.911521,30.269076,"г.Санкт-Петербург, Курляндская улица, дом 47, ...",Администрация Адмиралтейского района,Заместитель главы администрации,Степанов Алексей Викторович,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60876,Невский,3375243,2021-04-22 17:10:28.180799,Антон П.,59.881085,30.438858,"г.Санкт-Петербург, улица Бабушкина, дом 36, ли...",СПб ГКУ ЖА Невского района,Директор,Середин Владимир Ипполитович,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60877,Центральный,3375329,2021-04-23 09:43:33.320180,Антон К.,59.941825,30.323319,Неизвестно,Администрация Центрального района,Заместитель главы администрации,Долгополый Михаил Николаевич,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60878,Невский,3375365,2021-04-23 10:00:41.923295,Артём Н.,59.880117,30.466739,"г.Санкт-Петербург, Народная улица, дом 25, лит...",СПб ГКУ ЖА Невского района,Директор,Середин Владимир Ипполитович,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60879,Невский,3375863,2021-04-23 09:32:27.875504,Людмила К.,59.830989,30.508650,Неизвестно,СПб ГКУ ЖА Невского района,Директор,Середин Владимир Ипполитович,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df2.isna().sum()

district_name                   0
sidebar.id                      0
payload.dt                      0
sidebar.author_name             0
sidebar.latitude                0
                               ..
январь управлять                0
январь управлять организация    0
ярослав                         0
ярослав гашек                   0
яхтенный                        0
Length: 15359, dtype: int64

### 2.3. Подготовка отчета

#### В ходе работы было выполнено:
* 2.1. Предварительная обработка данных - Обработка данных с помощью методов NLP: Токенизация; Удаление стоп-слов; Удаление пунктуации; Удаление спецсимволов; Лемматизация;
* 2.2. Поиск ключевых слов/биграмм/триграмм - Бало проаналищированно 3 алгоритма векторизации (TfidfVectorizer, HashingVectorizer, CountVectorizer); Были простроенны нграммы по алгоритму векторизации TfidfVectorizer;

In [20]:
#Сохранение набора данных
df.to_csv("df.csv", encoding='utf-8-sig', index=False)
df2.to_csv("result_data_clean.csv", encoding='utf-8-sig', index=False)